In [1]:
# 패키지 설치
!pip install transformers
!pip install datasets
!pip install sentence_transformers

# 패키지 로드
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, BartModel,BartForConditionalGeneration
import math
import re
import logging
from datetime import datetime
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from nltk.translate.bleu_score import corpus_bleu
from transformers import pipeline, AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=662a6167c85139e17ead7d7b191d3ba096d802df9b

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_sts = torch.load("/content/drive/MyDrive/conference(18,19)/model.pt")
data = pd.read_csv('/content/drive/MyDrive/conference(18,19)/final_data.csv')

content = []
for i in range(len(data)):
    content.append(str(data['contents'][i]))
document_embeddings = model_sts.encode(content)

In [29]:
class fn():

    def __init__(self, text):
        self.text = re.sub("(<span class='quot[0-9]'>|\n\r\n|</span>|<br/>|<br />|([^0-9가-힣A-Za-z. ]))","",text)

    def __str__(self):
        return f"해당 클래스는 '{self.text}'라는 문장이 참인지, 거짓인지 판단합니다."

    def summarize(self):
        tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
        model_sm = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
        text = self.text.replace('\n', ' ')
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        if len(input_ids) > 1024:
            summary_text = '해당 기사 본문 내용이 너무 길어서 요약에 실패했습니다. 좀더 적은 양의 본문 내용을 뽑아주세요'
        else:
            summary_ids = model_sm.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
            summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

        return summary_text

    def sts(self):
        query_embedding = model_sts.encode(self.text)
        cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
        top_results = torch.topk(cos_scores, k=3)
        num_top_results = sum(top_results[0] > 0.45)

        cos_contents = []
        label_contents = []

        if num_top_results != 0:
            for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
                if score <= 0.5: break
                label = 'True' if data['label'][int(idx)] == 1 else 'False'
                cos_contents.append(content[idx])
                label_contents.append(label)
            return cos_contents, label_contents

        elif num_top_results == 0:
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"

    def nli(self, con_text):
        classifier = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=True)
        tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
        results = classifier(f'{self.text} {tokenizer.sep_token} {con_text}')[0]
        return results

    def fact(self):
        if self.sts() == "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다":
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"

        else:
            sts_list = []
            for i in self.sts()[0]:
                sts_list.append(i)

            label_list = []
            for i in self.sts()[1]:
                label_list.append(i)

            nli_list = []
            for i in sts_list:
                nli_list.append(self.nli(i))

            fact_list = []

            for i in range(len(nli_list)):
                score = []
                for j in nli_list[i]:
                    score.append(j['score'])

                if score[0]>score[2]:
                    if label_list[i] == 'True':
                        fact_list.append('True')
                    else:
                        fact_list.append('False')

                else:
                    if label_list[i] == 'True':
                        fact_list.append('False')
                    else:
                        fact_list.append('True')

            return fact_list

    def result(self):
        if self.fact() == "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다":
            return "입력 문장과 유사한 문장이 없으므로 확인이 불가능합니다"
        else:
            if len(self.fact()) == 1:
                return f"입력 문장'{self.text}'에 대한 진위 여부는 '{self.fact()}'입니다."
            elif len(self.fact()) == 2:
                if self.fact() == ['True','False'] or self.fact() == ['False','True']:
                    return f"입력 문장'{self.text}'에 대한 판단을 유보합니다"
                else:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 '{self.fact()[0]}'입니다."

            else:
                t = 0
                f = 0
                for i in self.fact():
                    if i == 'True':
                        t += 1
                    else:
                        f += 1
                if t>f:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 'True'입니다."
                else:
                    return f"입력 문장'{self.text}'에 대한 진위 여부는 'False'입니다."



In [30]:
a = fn("문재인 아들이 비리 의혹이 없다")

In [8]:
a.sts()

(['안철수 문재인 아들 5급 공무원에 특채는 사실', '문재인 민주당 후보 아들이 고용정보원에서 휴직 중 승진했다 는 의혹 '],
 ['False', 'False'])

In [9]:
a.nli('안철수 문재인 아들 5급 공무원에 특채는 사실')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'ENTAILMENT', 'score': 0.00031106884125620127},
 {'label': 'NEUTRAL', 'score': 0.9961093068122864},
 {'label': 'CONTRADICTION', 'score': 0.003579586511477828}]

In [ ]:
a.nli('문재인 민주당 후보 아들이 고용정보원에서 휴직 중 승진했다 는 의혹')

[{'label': 'ENTAILMENT', 'score': 0.00038911771844141185},
 {'label': 'NEUTRAL', 'score': 0.9987987279891968},
 {'label': 'CONTRADICTION', 'score': 0.0008122034487314522}]

In [17]:
f = a.fact()

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [20]:
f[0]

'True'

In [31]:
r = a.result()

In [32]:
r

"입력 문장'문재인 아들이 비리 의혹이 없다'에 대한 진위 여부는 'True'입니다."